In [1]:
# https://arxiv.org/abs/cond-mat/0703314
# https://arxiv.org/abs/0808.3779

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

In [3]:
@jit(nopython=True)
def calc_rsks(L):
    rs = np.arange(1,L+1)
    ks = np.linspace(np.pi/(L+1.0),L*np.pi/(L+1.0),L)
    return rs,ks

@jit(nopython=True)
def calc_eks(J,ks):
    return -2.0*J*np.cos(ks)

@jit(nopython=True)
def calc_xkrs(L,ks,rs):
    return np.sqrt(2.0/(L+1.0))*np.sin(np.outer(ks,rs))

# #@jit(nopython=True)
# def calc_ys(t,eks,xkrs):
#     expeks = np.exp(1j*t*eks)
#     return np.einsum("ik,k,kj->ij",xkrs,expeks,xkrs)

@jit(nopython=True)
def calc_ys(t,eks,xkrs):
    xc = (1.0+0.0j)*xkrs # for numba
    expeks = np.diag(np.exp(1j*t*eks))
    return xc @ expeks @ xc

@jit(nopython=True)
def calc_fs(ys,state):
    diag = (1.0+0.0j)*np.diag(state)
    return ys @ diag @ ys.T.conjugate()

In [4]:
L = 2
J = 1.0
t = 0.1

rs, ks = calc_rsks(L)
print("rs",rs)
print("ks",ks)

eks = calc_eks(J,ks)
print("eks",eks)

xkrs = calc_xkrs(L,ks,rs)
print("xkrs",xkrs)
print("xkrs.T",xkrs.T)
print("xkrs~xkrs.T",np.allclose(xkrs,xkrs.T))

ys = calc_ys(t,eks,xkrs)
print("ys",ys)

state1 = np.ones(L,dtype=np.float64)
print("state1",state1)
fs1 = calc_fs(ys,state1)
print("fs1",fs1)
print("fs1~I",np.allclose(fs1,np.eye(L)))

state2 = np.array([1.0 if i%2==0 else 0.0 for i in range(L)])
print("state2",state2)
fs2 = calc_fs(ys,state2)
print("fs2",fs2)

rs [1 2]
ks [1.04719755 2.0943951 ]
eks [-1.  1.]
xkrs [[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]
xkrs.T [[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]
xkrs~xkrs.T True
ys [[9.95004165e-01-1.62625244e-18j 1.14545220e-17-9.98334166e-02j]
 [1.26311104e-17-9.98334166e-02j 9.95004165e-01-7.94590319e-17j]]
state1 [1. 1.]
fs1 [[1.00000000e+00+2.05104213e-37j 3.20603256e-17+1.38777878e-17j]
 [3.20603256e-17-1.38777878e-17j 1.00000000e+00+4.49162938e-33j]]
fs1~I True
state2 [1. 0.]
fs2 [[9.90033289e-01-8.44236020e-35j 1.27303618e-17+9.93346654e-02j]
 [1.27303618e-17-9.93346654e-02j 9.96671108e-03+9.30157441e-35j]]


In [5]:
L = 4
J = 1.0
t = 0.1

rs, ks = calc_rsks(L)
print("rs",rs)
print("ks",ks)

eks = calc_eks(J,ks)
print("eks",eks)

xkrs = calc_xkrs(L,ks,rs)
print("xkrs",xkrs)
print("xkrs.T",xkrs.T)
print("xkrs~xkrs.T",np.allclose(xkrs,xkrs.T))

ys = calc_ys(t,eks,xkrs)
print("ys",ys)

state1 = np.ones(L,dtype=np.float64)
print("state1",state1)
fs1 = calc_fs(ys,state1)
print("fs1",fs1)
print("fs1~I",np.allclose(fs1,np.eye(L)))

state2 = np.array([1.0 if i%2==0 else 0.0 for i in range(L)])
print("state2",state2)
fs2 = calc_fs(ys,state2)
print("fs2",fs2)

rs [1 2 3 4]
ks [0.62831853 1.25663706 1.88495559 2.51327412]
eks [-1.61803399 -0.61803399  0.61803399  1.61803399]
xkrs [[ 0.37174803  0.60150096  0.60150096  0.37174803]
 [ 0.60150096  0.37174803 -0.37174803 -0.60150096]
 [ 0.60150096 -0.37174803 -0.37174803  0.60150096]
 [ 0.37174803 -0.60150096  0.60150096 -0.37174803]]
xkrs.T [[ 0.37174803  0.60150096  0.60150096  0.37174803]
 [ 0.60150096  0.37174803 -0.37174803 -0.60150096]
 [ 0.60150096 -0.37174803 -0.37174803  0.60150096]
 [ 0.37174803 -0.60150096  0.60150096 -0.37174803]]
xkrs~xkrs.T True
ys [[ 9.95008326e-01+3.55174960e-18j -2.40712241e-17-9.96670831e-02j
  -4.98751111e-03-6.09936006e-18j  8.19534984e-17+1.66416825e-04j]
 [-2.67633433e-17-9.96670831e-02j  9.90020815e-01-1.23010111e-18j
   2.63362170e-17-9.95006663e-02j -4.98751111e-03-1.42259991e-17j]
 [-4.98751111e-03-1.33063497e-17j  2.72084733e-17-9.95006663e-02j
   9.90020815e-01-2.72746462e-17j  2.07033967e-16-9.96670831e-02j]
 [ 8.33012086e-17+1.66416825e-04j -4.987511

In [6]:
L = 6
J = 1.0
t = 0.1

rs, ks = calc_rsks(L)
print("rs",rs)
print("ks",ks)

eks = calc_eks(J,ks)
print("eks",eks)

xkrs = calc_xkrs(L,ks,rs)
print("xkrs",xkrs)
print("xkrs.T",xkrs.T)
print("xkrs~xkrs.T",np.allclose(xkrs,xkrs.T))

ys = calc_ys(t,eks,xkrs)
print("ys",ys)

state1 = np.ones(L,dtype=np.float64)
print("state1",state1)
fs1 = calc_fs(ys,state1)
print("fs1",fs1)
print("fs1~I",np.allclose(fs1,np.eye(L)))

state2 = np.array([1.0 if i%2==0 else 0.0 for i in range(L)])
print("state2",state2)
fs2 = calc_fs(ys,state2)
print("fs2",fs2)

rs [1 2 3 4 5 6]
ks [0.44879895 0.8975979  1.34639685 1.7951958  2.24399475 2.6927937 ]
eks [-1.80193774 -1.2469796  -0.44504187  0.44504187  1.2469796   1.80193774]
xkrs [[ 0.23192061  0.41790651  0.52112089  0.52112089  0.41790651  0.23192061]
 [ 0.41790651  0.52112089  0.23192061 -0.23192061 -0.52112089 -0.41790651]
 [ 0.52112089  0.23192061 -0.41790651 -0.41790651  0.23192061  0.52112089]
 [ 0.52112089 -0.23192061 -0.41790651  0.41790651  0.23192061 -0.52112089]
 [ 0.41790651 -0.52112089  0.23192061  0.23192061 -0.52112089  0.41790651]
 [ 0.23192061 -0.41790651  0.52112089 -0.52112089  0.41790651 -0.23192061]]
xkrs.T [[ 0.23192061  0.41790651  0.52112089  0.52112089  0.41790651  0.23192061]
 [ 0.41790651  0.52112089  0.23192061 -0.23192061 -0.52112089 -0.41790651]
 [ 0.52112089  0.23192061 -0.41790651 -0.41790651  0.23192061  0.52112089]
 [ 0.52112089 -0.23192061 -0.41790651  0.41790651  0.23192061 -0.52112089]
 [ 0.41790651 -0.52112089  0.23192061  0.23192061 -0.52112089  0.417906